In [1]:
import pandas as pd
import requests
from tqdm.auto import tqdm
tqdm.pandas()

In [2]:
def next_monday(date):
    return pd.date_range(start=date, end=date + pd.offsets.Day(6), freq='W-MON')[0]

# Setting

In [3]:
test_date = pd.to_datetime('2021-03-22')

In [3]:
test_date = pd.to_datetime('2021-05-03')

In [5]:
models_to_exclude = ['COVIDhub-ensemble', 'COVIDhub-trained_ensemble', 
                     'CU-nochange', 'CU-scenario_high', 'CU-scenario_low', 'CU-scenario_mid']

locations_to_exclude = ["60", "66", "69", "72", "74", "78"]

# DC,11,District of Columbia
# AS,60,American Samoa
# GU,66,Guam
# MP,69,Northern Mariana Islands
# PR,72,Puerto Rico
# UM,74,U.S. Minor Outlying Islands
# VI,78,Virgin Islands

In [6]:
dtype={'target': str, 'location': str, 'type': str, 'quantile': float, 'value': float}
parse_dates=['forecast_date', 'target_end_date']

# Load Files

In [7]:
url = "https://api.github.com/repos/reichlab/covid19-forecast-hub/git/trees/master?recursive=1"
r = requests.get(url)
res = r.json()

In [8]:
files = [file["path"] for file in res["tree"] if (file["path"].startswith('data-processed/') and file["path"].endswith('.csv'))]

In [9]:
files[-5:]

['data-processed/epiforecasts-ensemble1/2021-04-05-epiforecasts-ensemble1.csv',
 'data-processed/epiforecasts-ensemble1/2021-04-12-epiforecasts-ensemble1.csv',
 'data-processed/epiforecasts-ensemble1/2021-04-19-epiforecasts-ensemble1.csv',
 'data-processed/epiforecasts-ensemble1/2021-04-26-epiforecasts-ensemble1.csv',
 'data-processed/epiforecasts-ensemble1/2021-05-03-epiforecasts-ensemble1.csv']

In [10]:
df_files = pd.DataFrame({'filename':files})

df_files['model'] = df_files.filename.apply(lambda f: f.split('/')[1])

df_files['forecast_date'] = df_files.filename.apply(lambda f: f.split('/')[2][:10])
df_files.forecast_date = pd.to_datetime(df_files.forecast_date)

df_files['timezero'] = df_files.forecast_date.apply(next_monday)

df_files = df_files[~df_files.model.isin(models_to_exclude)]

In [11]:
df_files

,filename,model,forecast_date,timezero
0,data-processed/AIpert-pwllnod/2020-12-21-AIper...,AIpert-pwllnod,2020-12-21,2020-12-21
1,data-processed/AIpert-pwllnod/2020-12-28-AIper...,AIpert-pwllnod,2020-12-28,2020-12-28
2,data-processed/AIpert-pwllnod/2021-01-04-AIper...,AIpert-pwllnod,2021-01-04,2021-01-04
3,data-processed/AIpert-pwllnod/2021-01-11-AIper...,AIpert-pwllnod,2021-01-11,2021-01-11
4,data-processed/AIpert-pwllnod/2021-01-18-AIper...,AIpert-pwllnod,2021-01-18,2021-01-18
...,...,...,...,...
3296,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-05,2021-04-05
3297,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-12,2021-04-12
3298,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-19,2021-04-19
3299,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-26,2021-04-26


### Test Data

In [12]:
VALID_TARGETS = [f"{_} wk ahead inc death" for _ in range(1, 5)] + \
                [f"{_} wk ahead cum death" for _ in range(1, 5)] + \
                [f"{_} wk ahead inc case" for _ in range(1, 5)]

In [13]:
df_test_files = df_files[df_files.timezero == test_date]

In [14]:
df_test_files.head()

,filename,model,forecast_date,timezero
19,data-processed/AIpert-pwllnod/2021-05-03-AIper...,AIpert-pwllnod,2021-05-03,2021-05-03
62,data-processed/BPagano-RtDriven/2021-05-02-BPa...,BPagano-RtDriven,2021-05-02,2021-05-03
108,data-processed/CEID-Walk/2021-05-03-CEID-Walk.csv,CEID-Walk,2021-05-03,2021-05-03
150,data-processed/CMU-TimeSeries/2021-05-03-CMU-T...,CMU-TimeSeries,2021-05-03,2021-05-03
207,data-processed/COVIDhub-baseline/2021-05-03-CO...,COVIDhub-baseline,2021-05-03,2021-05-03


In [15]:
dfs_test = []

for _, row in tqdm(df_test_files.iterrows(), total=df_test_files.shape[0]):
    #print(row['filename'])
    df_temp = pd.read_csv('https://github.com/reichlab/covid19-forecast-hub/raw/master/' + row['filename'],
                         dtype=dtype, parse_dates=parse_dates)
    df_temp = df_temp[df_temp.target.isin(VALID_TARGETS)]
    df_temp['model'] = row['model']
    dfs_test.append(df_temp)

  0%|          | 0/53 [00:00<?, ?it/s]

In [16]:
df_test = pd.concat(dfs_test)

In [17]:
df_test.shape

(2154757, 8)

In [19]:
df_test

,forecast_date,target,target_end_date,quantile,type,value,location,model
0,2021-05-03,1 wk ahead cum death,2021-05-08,0.025,quantile,580229.206498,US,AIpert-pwllnod
1,2021-05-03,1 wk ahead cum death,2021-05-08,0.250,quantile,581038.673139,US,AIpert-pwllnod
2,2021-05-03,1 wk ahead cum death,2021-05-08,0.750,quantile,582321.629640,US,AIpert-pwllnod
3,2021-05-03,1 wk ahead cum death,2021-05-08,0.975,quantile,583957.272774,US,AIpert-pwllnod
4,2021-05-03,1 wk ahead cum death,2021-05-08,NaN,point,581463.402736,US,AIpert-pwllnod
...,...,...,...,...,...,...,...,...
10939,2021-05-03,4 wk ahead cum death,2021-05-29,0.850,quantile,599412.000000,US,epiforecasts-ensemble1
10940,2021-05-03,4 wk ahead cum death,2021-05-29,0.900,quantile,600159.000000,US,epiforecasts-ensemble1
10941,2021-05-03,4 wk ahead cum death,2021-05-29,0.950,quantile,601476.000000,US,epiforecasts-ensemble1
10942,2021-05-03,4 wk ahead cum death,2021-05-29,0.975,quantile,602612.000000,US,epiforecasts-ensemble1


In [20]:
# only consider US + 50 states
df_test = df_test[df_test.location.str.len() == 2]
df_test = df_test[~df_test.location.isin(locations_to_exclude)]

In [21]:
df_test.shape

(337726, 8)

In [22]:
df_test.location.nunique()

52

In [23]:
# ensure that for all targets each model provides forecasts for all locations
df_test = df_test[df_test.groupby(['target', 'model'])['location'].transform('nunique') == 52]

In [24]:
df_test = df_test[df_test.type == 'quantile']

In [25]:
df_test['no_quantiles'] = df_test.groupby(['model', 'target', 'target_end_date', 'location'])['quantile'].transform('nunique')

df_test['no_quantiles'] = df_test.groupby(['target', 'model'])['no_quantiles'].transform('min')

In [26]:
df_test = df_test[(df_test.no_quantiles == 23) | 
        (df_test.target.str.contains('inc case') & (df_test.no_quantiles == 7))].drop(columns='no_quantiles').reset_index(drop=True)

In [27]:
df_test.shape

(242216, 8)

In [28]:
# dict of the models available for each target
available_models = dict(df_test.groupby(['target'])['model'].unique())

In [29]:
available_models

{'1 wk ahead cum death': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline', 'CU-select',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI', 'DDS-NBDS',
        'JHUAPL-Bucky', 'JHU_CSSE-DECOM', 'JHU_IDD-CovidSP', 'Karlen-pypm',
        'LANL-GrowthRate', 'LNQ-ens1', 'MIT_CritData-GBCF',
        'RobertWalraven-ESG', 'SteveMcConnell-CovidComplete',
        'UA-EpiCovDA', 'UCLA-SuEIR', 'UMass-MechBayes', 'USC-SI_kJalpha',
        'epiforecasts-ensemble1'], dtype=object),
 '1 wk ahead inc case': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline', 'CU-select',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI',
        'IEM_MED-CovidProject', 'JHUAPL-Bucky', 'JHU_CSSE-DECOM',
        'JHU_IDD-CovidSP', 'Karlen-pypm', 'LANL-GrowthRate', 'LNQ-ens1',
        'RobertWalraven-ESG', 'UCLA-SuEIR', 'UChicagoCHATTOPADHYAY-UnIT',
        'USC-SI_kJalpha', 'UVA-Ensemble'], dtype=object),
 '1 wk ahead inc death': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline'

## Training Data

d - 4 weeks - (horizon - 1) --> end: d - horizon

In [30]:
lower_bound = test_date - pd.Timedelta(weeks=4) - pd.Timedelta(weeks=(4 - 1))

In [31]:
lower_bound

Timestamp('2021-03-15 00:00:00')

In [32]:
df_files = df_files[(df_files.timezero >= lower_bound) & (df_files.timezero < test_date)].copy()

In [33]:
# assigns to each horizon the corresponding training forecast dates for the test date
h_dict = {}
for h in range(1, 5):
    h_dict[h] = [test_date - pd.Timedelta(weeks=4) - pd.Timedelta(weeks=(h - 1)), 
                 test_date - pd.Timedelta(weeks=h)]

In [34]:
h_dict

{1: [Timestamp('2021-04-05 00:00:00'), Timestamp('2021-04-26 00:00:00')],
 2: [Timestamp('2021-03-29 00:00:00'), Timestamp('2021-04-19 00:00:00')],
 3: [Timestamp('2021-03-22 00:00:00'), Timestamp('2021-04-12 00:00:00')],
 4: [Timestamp('2021-03-15 00:00:00'), Timestamp('2021-04-05 00:00:00')]}

In [35]:
def relevant_horizons(d):
    hs = []
    for h in range(1, 5):
        if((d >= h_dict[h][0]) & (d <= h_dict[h][1])):
            hs.append(h)
    return hs

In [36]:
df_files['horizons'] = df_files.timezero.apply(relevant_horizons)

In [37]:
df_files

,filename,model,forecast_date,timezero,horizons
12,data-processed/AIpert-pwllnod/2021-03-15-AIper...,AIpert-pwllnod,2021-03-15,2021-03-15,[4]
13,data-processed/AIpert-pwllnod/2021-03-22-AIper...,AIpert-pwllnod,2021-03-22,2021-03-22,"[3, 4]"
14,data-processed/AIpert-pwllnod/2021-03-29-AIper...,AIpert-pwllnod,2021-03-29,2021-03-29,"[2, 3, 4]"
15,data-processed/AIpert-pwllnod/2021-04-05-AIper...,AIpert-pwllnod,2021-04-05,2021-04-05,"[1, 2, 3, 4]"
16,data-processed/AIpert-pwllnod/2021-04-12-AIper...,AIpert-pwllnod,2021-04-12,2021-04-12,"[1, 2, 3]"
...,...,...,...,...,...
3295,data-processed/epiforecasts-ensemble1/2021-03-...,epiforecasts-ensemble1,2021-03-29,2021-03-29,"[2, 3, 4]"
3296,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-05,2021-04-05,"[1, 2, 3, 4]"
3297,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-12,2021-04-12,"[1, 2, 3]"
3298,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-19,2021-04-19,"[1, 2]"


In [38]:
# only keep relevant training data
df_files = df_files[df_files.horizons.apply(len) > 0]

In [39]:
dfs = []
for _, row in tqdm(df_files.iterrows(), total=df_files.shape[0]):
    VALID_TARGETS = [f"{_} wk ahead inc death" for _ in row['horizons']] + \
                    [f"{_} wk ahead cum death" for _ in row['horizons']] + \
                    [f"{_} wk ahead inc case" for _ in row['horizons']]
    df_temp = pd.read_csv('https://github.com/reichlab/covid19-forecast-hub/raw/master/' + row['filename'],
                         dtype=dtype, parse_dates=parse_dates)
    df_temp = df_temp[df_temp.target.isin(VALID_TARGETS)]
    df_temp['model'] = row['model']
    dfs.append(df_temp)

  0%|          | 0/429 [00:00<?, ?it/s]

In [40]:
df = pd.concat(dfs)

In [41]:
df.shape

(9271049, 8)

# Data Cleaning

In [42]:
df = df[df.location.str.len() == 2]
df = df[~df.location.isin(locations_to_exclude)]

In [44]:
df.location.nunique() # US + 50 states + district of Columbia

52

In [45]:
df = df[df.type == 'quantile']

In [46]:
# how many forecasts for each target/model/location? should be 4 for every location
df['no_forecasts'] = df.groupby(['target', 'model', 'location'])['target_end_date'].transform('nunique')

In [47]:
df = df[df.groupby(['target', 'model'])['no_forecasts'].transform('min') == 4].drop(columns='no_forecasts').reset_index(drop=True)

In [48]:
# df = df[df.groupby(['target', 'model', 'location'])['target_end_date'].transform('nunique') == 4]
# would only remove the respective location with <4 forecasts, but we want to remove the model as a whole if in one location <4

In [49]:
df = df[df.groupby(['target', 'model', 'target_end_date'])['location'].transform('nunique') == 52]

In [50]:
df['no_quantiles'] = df.groupby(['model', 'target', 'target_end_date', 'location'])['quantile'].transform('nunique')

df['no_quantiles'] = df.groupby(['target', 'model'])['no_quantiles'].transform('min')

In [51]:
df[(df.target.str.contains('inc case') & (df.no_quantiles == 7))]

,forecast_date,target,target_end_date,quantile,type,value,location,model,no_quantiles
9048,2021-03-14,4 wk ahead inc case,2021-04-10,0.025,quantile,149790.095050,US,BPagano-RtDriven,7
9049,2021-03-14,4 wk ahead inc case,2021-04-10,0.100,quantile,185585.761720,US,BPagano-RtDriven,7
9050,2021-03-14,4 wk ahead inc case,2021-04-10,0.250,quantile,218116.896790,US,BPagano-RtDriven,7
9051,2021-03-14,4 wk ahead inc case,2021-04-10,0.500,quantile,254380.906450,US,BPagano-RtDriven,7
9052,2021-03-14,4 wk ahead inc case,2021-04-10,0.750,quantile,290644.916110,US,BPagano-RtDriven,7
...,...,...,...,...,...,...,...,...,...
1103545,2021-04-26,1 wk ahead inc case,2021-05-01,0.250,quantile,281235.045639,US,UVA-Ensemble,7
1103546,2021-04-26,1 wk ahead inc case,2021-05-01,0.500,quantile,373176.036042,US,UVA-Ensemble,7
1103547,2021-04-26,1 wk ahead inc case,2021-05-01,0.750,quantile,465117.026446,US,UVA-Ensemble,7
1103548,2021-04-26,1 wk ahead inc case,2021-05-01,0.900,quantile,547866.785835,US,UVA-Ensemble,7


In [52]:
df = df[(df.no_quantiles == 23) | 
        (df.target.str.contains('inc case') & (df.no_quantiles == 7))].drop(columns='no_quantiles').reset_index(drop=True)

In [53]:
df.groupby('target').model.nunique()

target
1 wk ahead cum death    19
1 wk ahead inc case     16
1 wk ahead inc death    21
2 wk ahead cum death    19
2 wk ahead inc case     16
2 wk ahead inc death    21
3 wk ahead cum death    20
3 wk ahead inc case     17
3 wk ahead inc death    21
4 wk ahead cum death    22
4 wk ahead inc case     16
4 wk ahead inc death    22
Name: model, dtype: int64

In [54]:
# check if there are models used for training that are not available for the test date
train_models = dict(df.groupby('target').model.unique())

a = [(k, t) for k, v in train_models.items() for t in v]
b = [(k, t) for k, v in available_models.items() for t in v]

[i for i in a if i not in b]

[('1 wk ahead cum death', 'Microsoft-DeepSTIA'),
 ('1 wk ahead inc case', 'Microsoft-DeepSTIA'),
 ('1 wk ahead inc death', 'Columbia_UNC-SurvCon'),
 ('1 wk ahead inc death', 'Microsoft-DeepSTIA'),
 ('2 wk ahead cum death', 'Microsoft-DeepSTIA'),
 ('2 wk ahead inc case', 'Microsoft-DeepSTIA'),
 ('2 wk ahead inc death', 'Columbia_UNC-SurvCon'),
 ('2 wk ahead inc death', 'Microsoft-DeepSTIA'),
 ('3 wk ahead cum death', 'Microsoft-DeepSTIA'),
 ('3 wk ahead cum death', 'SigSci-TS'),
 ('3 wk ahead inc case', 'Microsoft-DeepSTIA'),
 ('3 wk ahead inc case', 'SigSci-TS'),
 ('3 wk ahead inc death', 'Microsoft-DeepSTIA'),
 ('3 wk ahead inc death', 'SigSci-TS'),
 ('4 wk ahead cum death', 'IHME-CurveFit'),
 ('4 wk ahead cum death', 'Microsoft-DeepSTIA'),
 ('4 wk ahead cum death', 'PSI-DRAFT'),
 ('4 wk ahead cum death', 'WalmartLabsML-LogForecasting'),
 ('4 wk ahead inc case', 'Microsoft-DeepSTIA'),
 ('4 wk ahead inc death', 'IHME-CurveFit'),
 ('4 wk ahead inc death', 'MITCovAlliance-SIR'),
 ('4 wk 

In [55]:
df = df[df.apply(lambda x: x.model in (available_models[x.target]), axis=1)]

In [56]:
df.groupby('target').model.nunique()

target
1 wk ahead cum death    18
1 wk ahead inc case     15
1 wk ahead inc death    19
2 wk ahead cum death    18
2 wk ahead inc case     15
2 wk ahead inc death    19
3 wk ahead cum death    18
3 wk ahead inc case     15
3 wk ahead inc death    19
4 wk ahead cum death    18
4 wk ahead inc case     15
4 wk ahead inc death    18
Name: model, dtype: int64

In [57]:
df.shape

(820924, 8)

In [58]:
models_by_target = pd.DataFrame([(i, k) for i,j in train_models.items() for k in j], 
                  columns=['target','model'])

In [59]:
models_by_target[models_by_target.target == '4 wk ahead cum death']

,target,model
170,4 wk ahead cum death,BPagano-RtDriven
171,4 wk ahead cum death,CEID-Walk
172,4 wk ahead cum death,COVIDhub-baseline
173,4 wk ahead cum death,CU-select
174,4 wk ahead cum death,Covid19Sim-Simulator
175,4 wk ahead cum death,CovidAnalytics-DELPHI
176,4 wk ahead cum death,DDS-NBDS
177,4 wk ahead cum death,IHME-CurveFit
178,4 wk ahead cum death,JHUAPL-Bucky
179,4 wk ahead cum death,Karlen-pypm


In [60]:
df = df[~df.model.isin(models_to_exclude)]

In [61]:
df

,forecast_date,target,target_end_date,quantile,type,value,location,model
0,2021-03-14,4 wk ahead inc death,2021-04-10,0.010,quantile,2924.25465,US,BPagano-RtDriven
1,2021-03-14,4 wk ahead inc death,2021-04-10,0.025,quantile,3334.89792,US,BPagano-RtDriven
2,2021-03-14,4 wk ahead inc death,2021-04-10,0.050,quantile,3701.99137,US,BPagano-RtDriven
3,2021-03-14,4 wk ahead inc death,2021-04-10,0.100,quantile,4131.84577,US,BPagano-RtDriven
4,2021-03-14,4 wk ahead inc death,2021-04-10,0.150,quantile,4423.75485,US,BPagano-RtDriven
...,...,...,...,...,...,...,...,...
914311,2021-04-26,1 wk ahead cum death,2021-05-01,0.850,quantile,576853.00000,US,epiforecasts-ensemble1
914312,2021-04-26,1 wk ahead cum death,2021-05-01,0.900,quantile,576957.00000,US,epiforecasts-ensemble1
914313,2021-04-26,1 wk ahead cum death,2021-05-01,0.950,quantile,577061.00000,US,epiforecasts-ensemble1
914314,2021-04-26,1 wk ahead cum death,2021-05-01,0.975,quantile,577132.00000,US,epiforecasts-ensemble1


In [62]:
available_train_models = dict(df.groupby(['target'])['model'].unique())

In [63]:
available_train_models

{'1 wk ahead cum death': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline', 'CU-select',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI', 'DDS-NBDS',
        'JHUAPL-Bucky', 'Karlen-pypm', 'LANL-GrowthRate', 'LNQ-ens1',
        'MIT_CritData-GBCF', 'RobertWalraven-ESG',
        'SteveMcConnell-CovidComplete', 'UA-EpiCovDA', 'UMass-MechBayes',
        'USC-SI_kJalpha', 'epiforecasts-ensemble1'], dtype=object),
 '1 wk ahead inc case': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline', 'CU-select',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI',
        'IEM_MED-CovidProject', 'JHUAPL-Bucky', 'JHU_CSSE-DECOM',
        'Karlen-pypm', 'LANL-GrowthRate', 'LNQ-ens1', 'RobertWalraven-ESG',
        'USC-SI_kJalpha', 'UVA-Ensemble'], dtype=object),
 '1 wk ahead inc death': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline', 'CU-select',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI', 'DDS-NBDS',
        'IEM_MED-CovidProject', 'JHUAPL-Bucky'

In [64]:
df_test.shape

(242216, 8)

In [65]:
df_test = df_test[df_test.apply(lambda x: x.model in (available_train_models[x.target]), axis=1)]

In [66]:
df_test.head()

,forecast_date,target,target_end_date,quantile,type,value,location,model
0,2021-05-02,1 wk ahead inc death,2021-05-08,0.010,quantile,2588.77597,US,BPagano-RtDriven
1,2021-05-02,1 wk ahead inc death,2021-05-08,0.025,quantile,2846.87793,US,BPagano-RtDriven
2,2021-05-02,1 wk ahead inc death,2021-05-08,0.050,quantile,3077.60748,US,BPagano-RtDriven
3,2021-05-02,1 wk ahead inc death,2021-05-08,0.100,quantile,3347.78423,US,BPagano-RtDriven
4,2021-05-02,1 wk ahead inc death,2021-05-08,0.150,quantile,3531.25808,US,BPagano-RtDriven


In [67]:
df.groupby(['target'])['model'].unique()

target
1 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
1 wk ahead inc case     [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
1 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
2 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
2 wk ahead inc case     [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
2 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
3 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
3 wk ahead inc case     [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
3 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
4 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
4 wk ahead inc case     [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
4 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
Name: model, dtype: object

# Export

In [69]:
df.to_csv('data/ensemble_data/df_train_{}.csv'.format(str(test_date.date())), index=False)

In [70]:
df_test.to_csv('data/ensemble_data/df_test_{}.csv'.format(str(test_date.date())), index=False)